In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.optim import lr_scheduler
import os
import cv2

In [2]:
train_csv = pd.read_csv("train.csv")
test_csv = pd.read_csv("test.csv")

In [3]:
train_csv.head()

,id,digit,letter,0,1,2,3,4,5,6,...,774,775,776,777,778,779,780,781,782,783
0,1,5,L,1,1,1,4,3,0,0,...,2,1,0,1,2,4,4,4,3,4
1,2,0,B,0,4,0,0,4,1,1,...,0,3,0,1,4,1,4,2,1,2
2,3,4,L,1,1,2,2,1,1,1,...,3,3,3,0,2,0,3,0,2,2
3,4,9,D,1,2,0,2,0,4,0,...,3,3,2,0,1,4,0,0,1,1
4,5,6,A,3,0,2,4,0,3,0,...,4,4,3,2,1,3,4,3,1,2


In [4]:
test_csv.head()

,id,letter,0,1,2,3,4,5,6,7,...,774,775,776,777,778,779,780,781,782,783
0,2049,L,0,4,0,2,4,2,3,1,...,2,0,4,2,2,4,3,4,1,4
1,2050,C,4,1,4,0,1,1,0,2,...,0,3,2,4,2,4,2,2,1,2
2,2051,S,0,4,0,1,3,2,3,0,...,1,3,2,0,3,2,3,0,1,4
3,2052,K,2,1,3,3,3,4,3,0,...,3,0,3,2,4,1,0,4,4,4
4,2053,W,1,0,1,1,2,2,1,4,...,4,3,1,4,0,2,1,2,3,4


In [5]:
def make_folder(directory):
    if not os.path.isdir(directory):
        os.mkdir(directory)
        
path_train = os.path.join(os.getcwd(), 'EMNIST/Train')
path_test = os.path.join(os.getcwd(), 'EMNIST/Test')

In [6]:
make_folder(path_train)
make_folder(path_test)

In [7]:
for i in range(len(train_csv)):
    index = train_csv.loc[i, 'id']
    image = train_csv.loc[i, '0':].values.reshape(28, 28).astype(float)

    train_image = os.path.join(path_train, '{0:05d}'.format(index) + '.jpg')
    cv2.imwrite(train_image, image)

In [8]:
for i in range(len(test_csv)):
    index = test_csv.loc[i, 'id']
    image = test_csv.loc[i, '0':].values.reshape(28, 28).astype(int)
    
    test_image = os.path.join(path_test, '{0:05d}'.format(index) + '.jpg')
    cv2.imwrite(test_image, image)

In [9]:
data_transforms = transforms.Compose([transforms.ToTensor(),])

In [10]:
# img_size = 64
# data_transforms = transforms.Compose{
#     transforms.Compose([
#         transforms.Resize([img_size, img_size]),
#         transforms.RandomHorizontalFlip(),
#         transforms.RandomRotation(20),
#         transforms.Grayscale(num_output_channels=1),
#         transforms.ToTensor(),
#         transforms.Normalize([0.5], [0.5])
#     ])
# }

In [21]:
data_dir = os.path.join(os.getcwd(), 'EMNIST')
emnist_train = dset.ImageFolder(os.path.join(data_dir), data_transforms)
emnist_test = dset.ImageFolder(os.path.join(data_dir), data_transforms)

train_data = torch.utils.data.DataLoader(emnist_train, batch_size=512, shuffle=True, drop_last=True)
test_data = torch.utils.data.DataLoader(emnist_test, batch_size=512, shuffle=False, drop_last=True)

In [22]:
# def custom_imshow(img):
#     img = img.numpy()
#     plt.imshow(np.transpose(img, (1, 2, 0)))
#     plt.show()

# def process():
#     cnt = 0
#     for batch_idx, (inputs, targets) in enumerate(train_data):
#         if cnt == 10:
#             break
#         custom_imshow(inputs[0])
#         cnt += 1

# process()

In [23]:
class customCNN(nn.Module):
    def __init__(self):
        super(customCNN, self).__init__()
        
        self.layer1 = self.conv_module(3, 16)
        self.layer2 = self.conv_module(16, 24)
        self.layer3 = self.conv_module(24, 32)
        self.layer4 = self.conv_module(32, 64)
        self.layer5 = self.conv_module(64, 128)
        self.gap = self.global_avg_pool(128, 10)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.gap(out)
        out = out.view(-1, 10)
        
        return out
    
    def conv_module(self, in_num, out_num):
        return nn.Sequential(
            nn.Conv2d(in_num, out_num, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(out_num),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=1)
        )
    
    def global_avg_pool(self, in_num, out_num):
        return nn.Sequential(
            nn.Conv2d(in_num, out_num, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(out_num),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1))
        )

In [24]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = customCNN().to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.002)

In [25]:
print(model)

customCNN(
  (layer1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=3, stride=1, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(16, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=3, stride=1, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(24, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=3, stride=1, padding=0, dilation=1, ceil_mode=False)
  )
  (layer4): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchN

In [26]:
loss_arr = []
for i in range(50):
    for j, [image, label] in enumerate(train_data):
        x = image.to(device)
        y_ = label.to(device)
        
        optimizer.zero_grad()
        output = model.forward(x)
        loss = loss_func(output, y_)
        loss.backward()
        optimizer.step()
        
        if j % 1000 == 0:
            print(loss)
            loss_arr.append(loss.cpu().detach().numpy())

tensor(2.3213, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(1.9046, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(1.7788, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(1.7302, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(1.6380, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(1.5762, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(1.5402, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(1.4492, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(1.3914, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(1.3073, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(1.2448, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(1.2082, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(1.1644, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(1.0957, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(1.1044, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(1.0036, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.9743, device='cuda:0', grad_fn=<NllLossBackward

In [28]:
correct = 0
total = 0

with torch.no_grad():
    for image, label in test_data:
        x = image.to(device)
        y_ = label.to(device)
        
        output = model.forward(x)
        _, output_index = torch.max(output, 1)
        total += label.size(0)
        correct += (output_index == y_).sum().float()
        
    print("Accuracy of Test Data: {}".format(100 * correct / total))

Accuracy of Test Data: 90.90909576416016


In [18]:
submission = pd.read_csv('submission.csv')
submission.digit = torch.cat(output).detach().cpu().numpy()
submission.to_csv('./result/CustomCNN_result.csv', index=False)

TypeError: cat(): argument 'tensors' (position 1) must be tuple of Tensors, not Tensor